In [1]:
"""Note: this nb is self-contained, just requires the one saved pkl model in the results folder.
The point is to see how the Emb-grams model does on unseen tokens at test-time.
Specifically, we check the top-interacting words that exist in the test set but not in the train set.
"""

%load_ext autoreload
%autoreload 2
import torch
from transformers import BertModel, DistilBertModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import datasets
import numpy as np
import os.path
from spacy.lang.en import English
from datasets import load_from_disk
import pickle as pkl
from sklearn.linear_model import LogisticRegressionCV
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm
import sys
sys.path.append('..')
import analyze_helper
import dvu
dvu.set_style()

import pandas as pd
from os.path import join as oj
import string
from typing import List
import data
import matplotlib.pyplot as plt
import seaborn as sns
import config
pd.set_option('display.max_rows', None)

# Load embeddings (model trained on whole dataset)

In [4]:
config.results_dir

'/scratch/embgrams/embedded-ngrams/results'

In [5]:
!ls '/scratch/embgrams/embedded-ngrams/results'

emotion		      fitted_results_aggregated.pkl  sst2
financial_phrasebank  rotten_tomatoes		     tweet_eval


In [6]:
rs = pd.read_pickle(oj(config.results_dir, 'rs_vary_ngrams_test.pkl'))

# rr, r_sem = analyze_helper.average_seeds(rs)
rs.head()

,checkpoint,ngrams,ngrams_test,subsample,all,norm,dataset,seed,layer,parsing,ignore_cache,padding,dataset_key_text,num_features,acc_train,acc_val
0,nateraw/bert-base-uncased-emotion,1,1,-1,all,,emotion,1,last_hidden_state_mean,,False,True,text,768,0.907875,0.874
1,nateraw/bert-base-uncased-emotion,1,2,-1,all,,emotion,1,last_hidden_state_mean,,False,True,text,768,0.907875,0.874
2,nateraw/bert-base-uncased-emotion,1,3,-1,all,,emotion,1,last_hidden_state_mean,,False,True,text,768,0.907875,0.874
3,nateraw/bert-base-uncased-emotion,1,4,-1,all,,emotion,1,last_hidden_state_mean,,False,True,text,768,0.907875,0.874
4,nateraw/bert-base-uncased-emotion,1,5,-1,all,,emotion,1,last_hidden_state_mean,,False,True,text,768,0.907875,0.874


In [7]:
rs.shape

(85, 16)

In [8]:
dset = 'emotion'
d = rs[rs.dataset == dset]

In [10]:
d[['ngrams', 'ngrams_test', 'acc_val']]

,ngrams,ngrams_test,acc_val
0,1,1,0.8740
1,1,2,0.8740
2,1,3,0.8740
3,1,4,0.8740
4,1,5,0.8740
5,1,6,0.8740
6,1,7,0.8740
7,2,1,0.8830
8,2,2,0.8830
9,2,3,0.8830
